In [1]:
%matplotlib widget

import random,threading,os,sys,asyncio
from time import sleep
import pandas as pd
import numpy as np
import ipympl
from ipywidgets import interactive
import matplotlib.pyplot as plt

sys.path.append(r"C:\projects\OpenVisus\build\RelWithDebInfo")
from OpenVisus import *

Starting OpenVisus C:\projects\OpenVisus\build\RelWithDebInfo\OpenVisus\__init__.py 3.8.6 (tags/v3.8.6:db45529, Sep 23 2020, 15:52:53) [MSC v.1927 64 bit (AMD64)] sys.version_info(major=3, minor=8, micro=6, releaselevel='final', serial=0) ...


In [2]:
db=LoadDataset('http://atlantis.sci.utah.edu/mod_visus?dataset=2kbit1')
logic_box=db.getLogicBox()
dims=[logic_box[1][I]-logic_box[0][I] for I in range(3)]

def ReadSlice(db,axis,offset,quality):
    if axis==0:
        data=db.read(x=[offset,offset+1], quality=quality)
        return data[:,:,0]
    elif axis==1:
        data=db.read(y=[offset,offset+1], quality=quality)
        return data[:,0,:]
    else:
        data=db.read(z=[offset,offset+1], quality=quality)
        return data[0,:,:]

In [3]:
%matplotlib widget

class ShowStatistics:
    
    # constructor
    def __init__(self,db,axis,quality):
        self.db=db
        self.axis=axis
        self.quality=quality
    
        # create the figure
        self.fig, self.axs = plt.subplots(2)
        self.fig.show(self.fig)
        
    # runLoop
    async def runLoop(self):
        
        logic_box=self.db.getLogicBox()
        dims=[logic_box[1][I]-logic_box[0][I] for I in range(3)]
        
        # statistics
        x=range(dims[self.axis])
        y=[0]*dims[self.axis]         

        last_data=None
        for offset in range(dims[self.axis]): 

            try:
                data=ReadSlice(self.db,self.axis,offset,self.quality)
                last_data=data
            except:
                # it could be I don't have samples for that resolution
                data=last_data

            # apply threshold
            data=(data>80)*255

            # count non zero pixels
            non_zeros=np.count_nonzero(data)
            y[offset]=non_zeros

            # refresh plot
            self.axs[0].clear()
            self.axs[1].clear()
            self.axs[0].plot(x,y)
            self.axs[1].imshow(data, origin='lower',cmap='gray')    
            plt.draw()
            await asyncio.sleep(0.01)
           
    # run
    def run(self):
        all_tasks = asyncio.all_tasks()
        for task in all_tasks: task.cancel()
        asyncio.create_task(self.runLoop());        
        
s=ShowStatistics(db,axis=0,quality=-15)
s.run()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …